In [1]:
import pandas as pd
import numpy as np
import json
from collections import defaultdict
import requests
import math
from pathlib import Path
import datetime

## Setup

In [2]:
data_folder = Path('../../data/state')

In [3]:
with open(data_folder/"usStates.json", 'r') as f:
    state_json = json.load(f)

In [4]:
state_json

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '01',
   'properties': {'name': 'Alabama',
    'population': 4903185,
    'beds': 19000,
    'density': 94.65},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-87.359296, 35.00118],
      [-85.606675, 34.984749],
      [-85.431413, 34.124869],
      [-85.184951, 32.859696],
      [-85.069935, 32.580372],
      [-84.960397, 32.421541],
      [-85.004212, 32.322956],
      [-84.889196, 32.262709],
      [-85.058981, 32.13674],
      [-85.053504, 32.01077],
      [-85.141136, 31.840985],
      [-85.042551, 31.539753],
      [-85.113751, 31.27686],
      [-85.004212, 31.003013],
      [-85.497137, 30.997536],
      [-87.600282, 30.997536],
      [-87.633143, 30.86609],
      [-87.408589, 30.674397],
      [-87.446927, 30.510088],
      [-87.37025, 30.427934],
      [-87.518128, 30.280057],
      [-87.655051, 30.247195],
      [-87.90699, 30.411504],
      [-87.934375, 30.657966],
      [-88.011052, 30.685351

### Change 'name' to 'statename'

In [5]:
for state in state_json["features"]:
    state["properties"]["statename"] = state["properties"]["name"]
    del state["properties"]["name"]

## Add state abbreviations

In [6]:
state_to_abbr = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

## Add state centroids

This is pretty awful but we actually do two runs here and grab two different sets of centroids. The first we have all 52, but they are slightly inaccurate. The 2nd set is accurate but some were corrupted and we only have 45 or so. It works so it's just not priority to fix right now 

In [7]:
with open(data_folder/"dataCovidState.json", 'r') as f:
    centroids_json = json.load(f)

In [8]:
for state in state_json["features"]:
    state_id = state["id"]
    state["properties"]["lat"] = centroids_json[state_id]["LAT"]
    state["properties"]["long"] = centroids_json[state_id]["LONG"]

In [9]:
with open(data_folder/"stateCentroids.json", 'r') as f:
    centroids_json = json.load(f)

In [10]:
for state in centroids_json['features']:
    name = state["properties"]["name"]
    long,lat = state["geometry"]["coordinates"]
    for state2 in state_json["features"]:
        if state2["properties"]["statename"] == name:
            state2["properties"]["lat"] = lat
            state2["properties"]["long"] = long
            break

## Add state abbreviation


In [11]:
for state in state_json["features"]:
    state["properties"]["abbr"] = state_to_abbr[state["properties"]["statename"]]

## Get and add covid data

In [12]:
api_url = "https://covid19.mathdro.id/api/countries/USA/confirmed"
state_covid = requests.get(api_url).json()

In [13]:
#includes ms which fromtimestamp doesnt accept so we cut it off
def get_str_from_timestamp(timestamp):
    timestamp = int(str(timestamp)[:-3])
    cur = datetime.datetime.fromtimestamp(timestamp)
    return cur.strftime('%#m/%d %#I:%M%p')

In [14]:
keys = ["deaths", "recovered", "active"]
for state1 in state_covid:
    if(state1 == {}): continue
    for state2 in state_json["features"]:
        if(state1["provinceState"] == state2["properties"]["statename"]):
            for key in keys:
                state2["properties"][key] = state1[key]
            state2["properties"]["cases"] = state1["confirmed"]
            state2["properties"]["time_cases_update"] = get_str_from_timestamp(state1["lastUpdate"])    


In [15]:
state_json["features"][0]

{'type': 'Feature',
 'id': '01',
 'properties': {'population': 4903185,
  'beds': 19000,
  'density': 94.65,
  'statename': 'Alabama',
  'lat': 32.78990682135675,
  'long': -86.82778271419518,
  'abbr': 'AL',
  'deaths': 0,
  'recovered': 0,
  'active': 78,
  'cases': 78,
  'time_cases_update': '3/19 7:43PM'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-87.359296, 35.00118],
    [-85.606675, 34.984749],
    [-85.431413, 34.124869],
    [-85.184951, 32.859696],
    [-85.069935, 32.580372],
    [-84.960397, 32.421541],
    [-85.004212, 32.322956],
    [-84.889196, 32.262709],
    [-85.058981, 32.13674],
    [-85.053504, 32.01077],
    [-85.141136, 31.840985],
    [-85.042551, 31.539753],
    [-85.113751, 31.27686],
    [-85.004212, 31.003013],
    [-85.497137, 30.997536],
    [-87.600282, 30.997536],
    [-87.633143, 30.86609],
    [-87.408589, 30.674397],
    [-87.446927, 30.510088],
    [-87.37025, 30.427934],
    [-87.518128, 30.280057],
    [-87.655051, 30.247195],
    [-87

## Get testing data

In [16]:
#https://covidtracking.com/api/states/info <- this api has info about where the data comes from
api_url = 'https://covidtracking.com/api/states'
state_covid = requests.get(api_url).json()

In [17]:
state_covid

[{'state': 'AK',
  'positive': 9,
  'negative': 504,
  'pending': None,
  'death': None,
  'total': 513,
  'lastUpdateEt': '3/19 16:30',
  'checkTimeEt': '3/20 01:09'},
 {'state': 'AL',
  'positive': 78,
  'negative': 28,
  'pending': None,
  'death': 0,
  'total': 106,
  'lastUpdateEt': '3/19 17:30',
  'checkTimeEt': '3/20 01:14'},
 {'state': 'AR',
  'positive': 62,
  'negative': 351,
  'pending': 146,
  'death': None,
  'total': 559,
  'lastUpdateEt': '3/19 15:31',
  'checkTimeEt': '3/20 01:16'},
 {'state': 'AS',
  'positive': 0,
  'negative': None,
  'pending': None,
  'death': 0,
  'total': 0,
  'lastUpdateEt': '3/19 00:00',
  'checkTimeEt': '3/20 03:07'},
 {'state': 'AZ',
  'positive': 44,
  'negative': 177,
  'pending': 131,
  'death': 0,
  'total': 352,
  'lastUpdateEt': '3/19 00:00',
  'checkTimeEt': '3/20 01:22'},
 {'state': 'CA',
  'positive': 1058,
  'negative': 8787,
  'pending': None,
  'death': 19,
  'total': 9845,
  'lastUpdateEt': '3/20 01:25',
  'checkTimeEt': '3/20 01

In [18]:
def format_test_time(test_time):
    d, h = test_time.split(' ')
    h, m = h.split(":")
    h = int(h)
    am_pm = "AM" if h <= 12 else "PM"
    if(am_pm) == "pm": h-=12
    return f"{d} {h}:{m}{am_pm}"

In [19]:
for state1 in state_covid:
    if(state1 == {}): continue
    for state2 in state_json["features"]:
        if(state1["state"] == state2["properties"]["abbr"]):
            state2["properties"]["time_tests_updated"] = format_test_time(state1["lastUpdateEt"])
            testing_keys = ["test_positive", "test_negative", "test_pending", "test_total"]
            for key in testing_keys:
                state2["properties"][key] = state1[key.split('_')[1]]

In [20]:
state_json["features"][0]

{'type': 'Feature',
 'id': '01',
 'properties': {'population': 4903185,
  'beds': 19000,
  'density': 94.65,
  'statename': 'Alabama',
  'lat': 32.78990682135675,
  'long': -86.82778271419518,
  'abbr': 'AL',
  'deaths': 0,
  'recovered': 0,
  'active': 78,
  'cases': 78,
  'time_cases_update': '3/19 7:43PM',
  'time_tests_updated': '3/19 17:30PM',
  'test_positive': 78,
  'test_negative': 28,
  'test_pending': None,
  'test_total': 106},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-87.359296, 35.00118],
    [-85.606675, 34.984749],
    [-85.431413, 34.124869],
    [-85.184951, 32.859696],
    [-85.069935, 32.580372],
    [-84.960397, 32.421541],
    [-85.004212, 32.322956],
    [-84.889196, 32.262709],
    [-85.058981, 32.13674],
    [-85.053504, 32.01077],
    [-85.141136, 31.840985],
    [-85.042551, 31.539753],
    [-85.113751, 31.27686],
    [-85.004212, 31.003013],
    [-85.497137, 30.997536],
    [-87.600282, 30.997536],
    [-87.633143, 30.86609],
    [-87.408589, 30.6

## Risks

### Local Risk

In [51]:
#111 is to convert degrees to kilometers
def get_distance(c0, c1):
    lat_dist = abs(c0[0])-abs(c1[0])
    lng_dist = abs(c0[1])-abs(c1[1])
    distance = 111 * math.sqrt(lat_dist**2 + lng_dist**2)
    return distance

In [52]:
def calc_state_local_risk(props):
    cases = props["active"]
    population = props.get("population", -1)
    if cases == 'NaN': cases = 0
    return cases/(population/100000) if population != -1 else -1

In [53]:
for state in state_json["features"]:
    state["properties"]["risk_local"] = calc_state_local_risk(state["properties"])

### Neighbor Risk

In [54]:
def get_state_all_neighbor_risk(props):
    risk_details = {}
    for state in state_json["features"]:
        neighbor_props = state["properties"]
        neighbor_risk = get_state_neighbor_risk(props, neighbor_props)
        if(neighbor_risk != {}): 
            risk_details[neighbor_props["abbr"]] = neighbor_risk
    props["risk_nearby"] = calc_state_neighbor_risk(risk_details)
    props["risk_total"] = props["risk_nearby"] + props["risk_local"]

In [55]:
def calc_state_neighbor_risk(risks):
    num_risks = len(risks)
    total_neighbor_risk = 0
    for risk in risks.values():
        distance, neighbor_cases, neighbor_pop = risk.values()
        neighbor_density = neighbor_cases/(neighbor_pop/100000)
        total_neighbor_risk += neighbor_density*(2**(-1*math.log2(float(distance))))
        print("RISK: ", total_neighbor_risk)
    return total_neighbor_risk

In [56]:
def get_state_neighbor_risk(props, neighbor_props):
    neighbor_centroid = [neighbor_props["lat"], neighbor_props["long"]]
    neighbor_cases = neighbor_props["active"]
    neighbor_pop = neighbor_props["population"]
    centroid = [props["lat"], props["long"]]
    distance = get_distance(centroid, neighbor_centroid)
    if(props["abbr"] == neighbor_props["abbr"] or neighbor_cases == "NaN"):
        return {}
    else:
        return {"DISTANCE":distance, "CASES":neighbor_cases, "POP":neighbor_pop}

In [57]:
for state in state_json["features"]:
    get_state_all_neighbor_risk(state["properties"])

RISK:  0.0001490827884989173
RISK:  0.00037293800449690747
RISK:  0.0034597529041028945
RISK:  0.004157087285151465
RISK:  0.006322394606896457
RISK:  0.008691741021965743
RISK:  0.01084127815658447
RISK:  0.0186862676604199
RISK:  0.021653327779931363
RISK:  0.028598302061163587
RISK:  0.028832855034532034
RISK:  0.02921632781324682
RISK:  0.03310172131084093
RISK:  0.03418816459069774
RISK:  0.0352869336003419
RISK:  0.03610311456277947
RISK:  0.03804682335042168
RISK:  0.05164846957630329
RISK:  0.053259326989994055
RISK:  0.054580685351114266
RISK:  0.0569789220483229
RISK:  0.059545180385785476
RISK:  0.060464848417920034
RISK:  0.06569497102916978
RISK:  0.06639753436281773
RISK:  0.06674223049438158
RISK:  0.06760526978235723
RISK:  0.06850550479325808
RISK:  0.07006123057133444
RISK:  0.07528174367196973
RISK:  0.0760589280118207
RISK:  0.0933301117776148
RISK:  0.09465460588503583
RISK:  0.09577642525545987
RISK:  0.09685237770598314
RISK:  0.0977393306637996
RISK:  0.09825044

RISK:  0.060615015826649916
RISK:  0.06175525262767932
RISK:  0.06278012916919275
RISK:  0.06405552661381556
RISK:  0.0670848726003423
RISK:  0.06799725464871141
RISK:  0.06839092412301717
RISK:  0.07636162401058644
RISK:  0.07640064467408961
RISK:  0.07769946151382864
RISK:  0.08092009494632926
RISK:  0.08095303999420371
RISK:  0.0009421602029739301
RISK:  0.0010779124744925424
RISK:  0.0012277077518629911
RISK:  0.0022146693502951426
RISK:  0.002733438762556261
RISK:  0.004142703136223102
RISK:  0.016972645154264613
RISK:  0.023980346577230888
RISK:  0.04495342096005348
RISK:  0.046068729112616065
RISK:  0.04787605340511796
RISK:  0.04807329670020704
RISK:  0.04836836822195372
RISK:  0.05049191682819477
RISK:  0.051185651665372256
RISK:  0.051882857782653655
RISK:  0.05233386943509842
RISK:  0.053198901837886746
RISK:  0.056841516568744435
RISK:  0.062023746396592916
RISK:  0.06591339838934429
RISK:  0.07734451754470985
RISK:  0.08032377635601018
RISK:  0.08106773604564457
RISK:  0.0

In [58]:
state_json["features"][0]

{'type': 'Feature',
 'id': '01',
 'properties': {'population': 4903185,
  'beds': 19000,
  'density': 94.65,
  'statename': 'Alabama',
  'lat': 32.78990682135675,
  'long': -86.82778271419518,
  'abbr': 'AL',
  'deaths': 0,
  'recovered': 0,
  'active': 78,
  'cases': 78,
  'time_cases_update': '3/19 7:43PM',
  'time_tests_updated': '3/19 17:30PM',
  'test_positive': 78,
  'test_negative': 28,
  'test_pending': None,
  'test_total': 106,
  'risk_local': 1.5908027129304727,
  'risk_nearby': 0.12333337521739396,
  'risk_total': 1.7141360881478667},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-87.359296, 35.00118],
    [-85.606675, 34.984749],
    [-85.431413, 34.124869],
    [-85.184951, 32.859696],
    [-85.069935, 32.580372],
    [-84.960397, 32.421541],
    [-85.004212, 32.322956],
    [-84.889196, 32.262709],
    [-85.058981, 32.13674],
    [-85.053504, 32.01077],
    [-85.141136, 31.840985],
    [-85.042551, 31.539753],
    [-85.113751, 31.27686],
    [-85.004212, 31.003013

## Export

In [59]:
state_json["features"][0]["properties"].keys()

dict_keys(['population', 'beds', 'density', 'statename', 'lat', 'long', 'abbr', 'deaths', 'recovered', 'active', 'cases', 'time_cases_update', 'time_tests_updated', 'test_positive', 'test_negative', 'test_pending', 'test_total', 'risk_local', 'risk_nearby', 'risk_total'])

In [60]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(NpEncoder, self).default(obj)

with open("../../data/states.json", 'w') as f:
    json.dump(state_json, f, cls=NpEncoder)

with open("../../data/stateData.js", 'w') as f:
    f.write("let stateData = ")
    json.dump(state_json, f, cls=NpEncoder)